In [49]:
#try different models here

In [50]:
# Allowed to make changes.
import collections
import json
import pandas as pd
import re
import string
import timeit
from ast import literal_eval
!pip install transformers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
# Allowed to make changes.

# Pre-processing cell. You can use this cell to pre-process input data or load
# your models.
model_name = "deepset/tinyroberta-squad2"  #choose model name
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name, device=device)
# Dummy code.
paragraph_ds = []
global_model = {}
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

In [52]:
# Allowed to make changes.
def get_theme_model(theme):
  # Here, you can load and return a model which is fine tuned for a theme.
  # In case, you only have a global model, you can return that.

  global_model= nlp
  return global_model

In [53]:
# Allowed to make changes.
theme_df= pd.read_csv('/content/train_input_paragraph.csv')
def pred_theme_ans(questions, theme_model, pred_out):
  theme = questions[0]["theme"]
  reqtheme_df= theme_df[theme_df['theme']==theme].reset_index(drop=True)
  for step,question in enumerate(questions):

    ans = {}
    ans["question_id"] = question["id"]
    ques= normalize_answer(question['question'])

    ans['paragraph_id']= -1
    ans['answers']= " " 

    #change threshold here
    threshold=0.65
    best_score=0
    flag=0
    for i in range(len(reqtheme_df)):
      context= reqtheme_df.loc[i,'paragraph']
      context= normalize_answer(context)
      p= theme_model(context=context, question=ques)
      
      if p['score']>best_score and p['score']>threshold:
        ans['answers']= p['answer']
        ans['paragraph_id']= reqtheme_df.loc[i,'id']
        best_score=p['score']

    pred_out.append(ans)

In [54]:
# NOT allowed to make changes. 

# All theme prediction.
questions = json.loads(pd.read_csv("train_input_question.csv").to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv("train_theme_interval.csv").to_json(orient="records"))
pred_out = []
theme_inf_time = {}
for theme_interval in theme_intervals:
  theme_ques = questions[int(theme_interval["start"])-1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.

pred_df = pd.DataFrame.from_records(pred_out)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv('train_output_prediction.csv', index=False)

KeyboardInterrupt: ignored

In [46]:
# NOT allowed to make changes. 

# def normalize_answer(s):
#   """Lower text and remove punctuation, articles and extra whitespace."""
#   def remove_articles(text):
#     regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
#     return re.sub(regex, ' ', text)
#   def white_space_fix(text):
#     return ' '.join(text.split())
#   def remove_punc(text):
#     exclude = set(string.punctuation)
#     return ''.join(ch for ch in text if ch not in exclude)
#   def lower(text):
#     return text.lower()
#   return white_space_fix(remove_articles(remove_punc(lower(s))))

# def get_tokens(s):
#   if not s: return []
#   return normalize_answer(s).split()

# def calc_f1(a_gold, a_pred):
#   gold_toks = get_tokens(a_gold)
#   pred_toks = get_tokens(a_pred)
#   common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
#   num_same = sum(common.values())
#   if len(gold_toks) == 0 or len(pred_toks) == 0:
#     # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
#     return int(gold_toks == pred_toks)
#   if num_same == 0:
#     return 0
#   precision = 1.0 * num_same / len(pred_toks)
#   recall = 1.0 * num_same / len(gold_toks)
#   f1 = (2 * precision * recall) / (precision + recall)
#   return f1

# def calc_max_f1(predicted, ground_truths):
#   max_f1 = 0
#   for ground_truth in ground_truths:
#     f1 = calc_f1(predicted, ground_truth)
#     max_f1 = max(max_f1, f1)
#   return max_f1

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(str(a_gold))
  pred_toks = get_tokens(str(a_pred))
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  if len(ground_truths)==0:
    max_f1= calc_f1(predicted, ground_truths)
  else:
    for ground_truth in ground_truths:
      f1 = calc_f1(predicted, ground_truth)
      max_f1 = max(max_f1, f1)
  return max_f1

In [47]:
# NOT allowed to make changes. 

# Evaluation methodology.
metrics = {}
pred = pd.read_csv("train_output_prediction.csv")
truth = pd.read_csv("train_ground_truth.csv")
truth.paragraph_id = truth.paragraph_id.apply(literal_eval)
truth.answers = truth.answers.apply(literal_eval)
questions = pd.read_csv("train_input_question.csv")
for idx in pred.index:
  q_id = pred["question_id"][idx]
  q_rows = questions.loc[questions['id'] == q_id].iloc[-1]
  theme = q_rows["theme"]
  predicted_paragraph = pred["paragraph_id"][idx]
  predicted_ans = pred["answers"][idx]
  
  if theme not in metrics.keys():
    metrics[theme] = {"true_positive": 0, "true_negative": 0, "total_predictions": 0, "f1_sum": 0}

  truth_row = truth.loc[truth['question_id'] == q_id].iloc[-1]
  truth_paragraph_id = [ int(i) for i in truth_row["paragraph_id"] ]
  if predicted_paragraph in truth_paragraph_id:
    # Increase TP for that theme.
    metrics[theme]["true_positive"] = metrics[theme]["true_positive"] + 1
  # -1 prediction in case there is no paragraph which can answer the query.
  if predicted_paragraph == -1 and truth_row["paragraph_id"] == []:
    # Increase TN.
    metrics[theme]["true_negative"] = metrics[theme]["true_negative"] + 1
  # Increase total predictions for that theme.
  metrics[theme]["total_predictions"] = metrics[theme]["total_predictions"] + 1
  f1 = calc_max_f1(predicted_ans, truth_row["answers"])
  
  metrics[theme]["f1_sum"] = metrics[theme]["f1_sum"] + f1

In [48]:
# NOT allowed to make changes. 

# Final score.
inf_time_threshold = 200.0 # milliseconds.
final_para_score = 0.0
final_qa_score = 0.0
count=0

# Weight would stay hidden from teams.
#theme_weights = {"Kubernetes": 0.5, "ChatGPT": 0.4, "Football world cup": 0.1}

for theme in metrics:
  inf_time_score = 1.0
  metric = metrics[theme]
  para_score = (metric["true_positive"] + metric["true_negative"]) / metric["total_predictions"] 
  qa_score = metric["f1_sum"] / metric["total_predictions"]
  avg_inf_time = theme_inf_time[theme] / metric["total_predictions"]
  if avg_inf_time > inf_time_threshold:
    inf_time_score = inf_time_threshold / avg_inf_time
  #final_qa_score += theme_weights[theme] * inf_time_score * qa_score
  final_qa_score += inf_time_score * qa_score
  #final_para_score += theme_weights[theme] * inf_time_score * para_score
  final_para_score += inf_time_score * para_score
  count+=1

print (final_para_score/count)
print (final_qa_score/count)

0.1659186109193456
0.1729102657250878
